[![imagenes](img/pythonista.png)](https://www.pythonista.io)

# Desarrollo de una servidor de *API REST* con *Flask-SQLAlchemy*.

En este capítulo se realizarán operaciones en la base de datos creada en la notebook [```20_la_extension_flask_sqlalchemy.ipynb```](20_la_extension_flask_sqlalchemy.ipynb).

**Advertencia:** Es necesario que se haya creado correctamente la base de datos localizada en [```data/alumnos.db```]([data/alumnos.db).

## Importación de módulos y datos.

In [ ]:
from flask import Flask, request, abort
from data import esquema_alumno, carreras
from json import loads
from jsonschema import validate, ValidationError
from flask_sqlalchemy import SQLAlchemy
from flask_restful import Api, Resource

In [ ]:
esquema_alumno

## Creación de la aplicación e inicio de la sesión con la base de datos.

In [ ]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///data/alumnos.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)
api = Api(app)

## Definición de la clase ```Alumno```.

In [ ]:
class Alumno(db.Model):
    __tablename__ = 'alumnos'
    cuenta = db.Column(db.Integer, primary_key=True)
    nombre = db.Column(db.String(50))
    primer_apellido = db.Column(db.String(50))
    segundo_apellido = db.Column(db.String(50))
    carrera = db.Column(db.String(50))
    semestre = db.Column(db.Integer)
    promedio = db.Column(db.Float)
    al_corriente = db.Column(db.Boolean)

## La función ```extrae_estado()```.

In [ ]:
def extrae_estado(obj, esquema):
    return  {campo: getattr(obj, campo)
             for campo in esquema['properties']}

## La función ```construye_instancia()```.

In [ ]:
def construye_instancia(candidato, Cls):
    obj = Cls()
    for campo in candidato:
        setattr(obj, campo, candidato[campo])
    return obj

## La función ```datos_validos()```.

Esta función fue modificada para poder gestionar objetos instanciados de ```Alumno``` y añadir registros a la base de datos.

In [ ]:
def valida_datos(cuenta, candidato, esquema):
    try:
        candidato['cuenta'] = int(cuenta)
        validate(candidato, esquema)
        if set(esquema["properties"]).issuperset(candidato):
            return candidato
        else:
            raise ValidationError("Invalid data")
    except ValidationError as e:
        abort(400, e)

In [ ]:
class Alumnos(Resource):
    def get(self):
        lista = Alumno.query.filter(Alumno.cuenta).all()
        return [extrae_estado(alumno, esquema_alumno) for alumno in lista]

In [ ]:
class AbcAlumnos(Resource):

    
    def get(self, cuenta):
        alumno = Alumno.query.filter_by(cuenta=cuenta).first_or_404()
        return extrae_estado(alumno, esquema_alumno)
            
            
    def delete(self, cuenta):
        alumno = Alumno.query.filter_by(cuenta=cuenta).first_or_404()
        db.session.delete(alumno)
        db.session.commit()
        return extrae_estado(alumno, esquema_alumno)
   

    def post(self, cuenta):
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno is None:
            data_alumno = valida_datos(cuenta, 
                                      loads(request.data),
                                      esquema_alumno)
            db.session.add(construye_instancia(data_alumno, Alumno))
            db.session.commit()
            return data_alumno
        else:
            abort(409)
            
            
    def put(self, cuenta):
        alumno = Alumno.query.filter_by(cuenta=cuenta).first_or_404()
        data_alumno = valida_datos(cuenta, 
                                    loads(request.data),
                                    esquema_alumno)
        db.session.delete(alumno)
        db.session.add(construye_instancia(data_alumno, Alumno))
        db.session.commit()
        return data_alumno

            
    def patch(self, cuenta):
        alumno = Alumno.query.filter_by(cuenta=cuenta).first_or_404()
        candidato = extrae_estado(alumno, esquema_alumno)
        candidato.update(loads(request.data))
        data_alumno = valida_datos(cuenta, 
                                    candidato,
                                    esquema_alumno)
        db.session.delete(alumno)
        db.session.add(construye_instancia(data_alumno, Alumno))
        db.session.commit()
        return data_alumno

In [ ]:
api.add_resource(Alumnos, '/api/')

In [ ]:
api.add_resource(AbcAlumnos, '/api/<int:cuenta>')

In [ ]:
app.run('0.0.0.0')

### Notas:

* **No reinicie o detenga el kernel de la notebook hasta que haya terminado todas las sesiones los clientes.**
* Debido a que el código de la celda de arriba levanta el servidor de Flask, ésta se ejecutará indefinidamente y desplegará los mensajes de respuesta a las peticiones de los clientes que se conecten.

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2022.</p>